# Introduction

This a short guide on how to generate *Plackett-Burman* designs for screening and computing main effects of factors using a linear model fit. For more information, check the [documentation](https://phrb.github.io/ExperimentalDesign.jl/dev/).

## Setup

First, check if you are at the correct project environment. It should be `ExperimentalDesign`:

In [6]:
using Pkg
Pkg.add("StatsModels")
Pkg.add("Distributions")
Pkg.add("Random")
Pkg.add("GLM")
Pkg.status()

  Resolving package versions...
julia version requirement for package `ExperimentalDesign [4babbea4]` not satisfied
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
julia version requirement for package `ExperimentalDesign [4babbea4]` not satisfied
   Updating `~/.julia/environments/v1.4/Project.toml`
  [31c24e10] + Distributions v0.22.6
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
julia version requirement for package `ExperimentalDesign [4babbea4]` not satisfied
   Updating `~/.julia/environments/v1.4/Project.toml`
  [9a3f8284] + Random 
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
julia version requirement for package `ExperimentalDesign [4babbea4]` not satisfied


Status `~/.julia/environments/v1.4/Project.toml`
  [a93c6f00] DataFrames v0.20.2
  [31c24e10] Distributions v0.22.6
  [4babbea4] ExperimentalDesign v0.2.0 [`~/.julia/dev/ExperimentalDesign`]
  [38e38edf] GLM v1.3.9
  [7073ff75] IJulia v1.21.2
  [47be7bcc] ORCA v0.3.1
  [f0f68f2c] PlotlyJS v0.13.1
  [91a5bcdd] Plots v1.2.2
  [3eaba693] StatsModels v0.6.11
  [9a3f8284] Random 


   Updating `~/.julia/environments/v1.4/Project.toml`
  [38e38edf] + GLM v1.3.9
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


Then check if all packages are installed and up to date:

In [2]:
Pkg.update()

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

julia version requirement for package `ExperimentalDesign [4babbea4]` not satisfied
  Installed LAME_jll ──────── v3.100.0+1
  Installed URIParser ─────── v0.4.1
  Installed PlotUtils ─────── v1.0.2
  Installed RecipesPipeline ─ v0.1.9
  Installed GeometryTypes ─── v0.8.3
  Installed Plots ─────────── v1.2.2
  Installed Blink ─────────── v0.12.3
  Installed ColorTypes ────── v0.10.3
  Installed libfdk_aac_jll ── v0.1.6+2
  Installed x265_jll ──────── v3.0.0+1
  Installed BinDeps ───────── v1.0.1
  Installed FFMPEG_jll ────── v4.1.0+3
  Installed Mux ───────────── v0.7.0
  Installed libass_jll ────── v0.14.0+2
  Installed Opus_jll ──────── v1.3.1+1
  Installed libvorbis_jll ─── v1.3.6+4
  Installed FriBidi_jll ───── v1.0.5+3
  Installed HTTP ──────────── v0.8.14
  Installed Ogg_jll ───────── v1.3.4+0
  Installed StaticArrays ──── v0.12.3
  Installed Lazy ──────────── v0.15.0
  Installed x264_jll ──────── v2019.5.25+2
  Installed GR ────────────── v0.49.1
################################

In [7]:
using ExperimentalDesign, StatsModels, GLM, DataFrames, Distributions, Random

# Generating Plackett-Burman Designs

A Plackett-Burman design is an orthogonal design matrix for factors $f_1,\dots,f_N$. Factors are encoded by high and low values, which can be mapped to the interval $[-1, 1]$. For designs in this package, the design matrix is a `DataFrame` from the [DataFrame package](https://juliastats.org/GLM.jl/stable/). For example, let's create a Plackett-Burman design for 16 factors:

In [52]:
design = PlackettBurman(16)
design.matrix

,factor1,factor2,factor3,factor4,factor5,factor6,factor7,factor8,factor9
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,1,1,1,1,1,1,1
2,1,1,-1,-1,-1,-1,1,-1,1
3,1,-1,-1,-1,-1,1,-1,1,-1
4,-1,-1,-1,-1,1,-1,1,-1,1
5,-1,-1,-1,1,-1,1,-1,1,1
6,-1,-1,1,-1,1,-1,1,1,1
7,-1,1,-1,1,-1,1,1,1,1
8,1,-1,1,-1,1,1,1,1,-1
9,-1,1,-1,1,1,1,1,-1,-1


Note that it is not possible to construct exact Plackett-Burman designs for all numbers of factors. In the example above, we needed a seventh extra "dummy" column to construct the design for six factors.

Using the `PlackettBurman` constructor enables quick construction of minimal screening designs for scenarios where we ignore interactions. We can access the underlying formula, which is a `Term` object from the [StatsModels package](https://juliastats.org/StatsModels.jl/stable/):

In [53]:
function generate_command(line, flags)
    cmd = "cargo build"
    for i in 1:length(flags)
        if line[i] > 0
            cmd = "-C passes=" * flags[i] * " " * cmd
        end
    end
    cmd = "RUSTFLAGS=-C lto=thin -C no-prepopulate-passes -C passes=name-anon-globals" * cmd
    return cmd
end

flags = ["constprop", "instcombine", "argpromotion", "jump-threading", "lcssa", "licm", "loop-deletion", "loop-extract", "loop-reduce", "loop-rotate", "loop-simplify", "loop-unroll", "loop-unroll-and-jam", "loop-unswitch", "mem2reg", "memcpyopt"]

generate_command.(eachrow(design.matrix[:, :]), repeat([flags], 20))

20-element Array{String,1}:
 "RUSTFLAGS=-C lto=thin -C no-prepopulate-passes -C passes=name-anon-globals-C passes=memcpyopt -C passes=mem2reg -C passes=loop-unswitch -C passes=loop-unroll-and-jam -C passes=loop-unroll -C passes=loop-simplify -C passes=loop-rotate -C passes=loop-reduce -C passes=loop-extract -C passes=loop-deletion -C passes=licm -C passes=lcssa -C passes=jump-threading -C passes=argpromotion -C passes=instcombine -C passes=constprop cargo build"
 "RUSTFLAGS=-C lto=thin -C no-prepopulate-passes -C passes=name-anon-globals-C passes=loop-unswitch -C passes=loop-unroll-and-jam -C passes=loop-unroll -C passes=loop-simplify -C passes=loop-reduce -C passes=loop-deletion -C passes=instcombine -C passes=constprop cargo build"
 "RUSTFLAGS=-C lto=thin -C no-prepopulate-passes -C passes=name-anon-globals-C passes=memcpyopt -C passes=loop-unroll-and-jam -C passes=loop-unroll -C passes=loop-simplify -C passes=loop-rotate -C passes=loop-extract -C passes=licm -C passes=constprop carg

In [54]:
println(design.formula)

response ~ -1 + factor1 + factor2 + factor3 + factor4 + factor5 + factor6 + factor7 + factor8 + factor9 + factor10 + factor11 + factor12 + factor13 + factor14 + factor15 + factor16 + dummy1 + dummy2 + dummy3


Notice we ignore interactions and include the dummy factor in the model. Strong main effects attributed to dummy factors may indicate important interactions.

We can obtain a tuple with the names of dummy factors:

In [55]:
design.dummy_factors

(:dummy1, :dummy2, :dummy3)

We can also get the main factors tuple:

In [56]:
design.factors

(:factor1, :factor2, :factor3, :factor4, :factor5, :factor6, :factor7, :factor8, :factor9, :factor10, :factor11, :factor12, :factor13, :factor14, :factor15, :factor16)

You can check other constructors on [the docs](https://phrb.github.io/ExperimentalDesign.jl/dev/lib/public/#ExperimentalDesign.PlackettBurman-Tuple{Int64}).

# Computing Main Effects

Suppose that the response variable on the experiments specified in our screening design is computed by:

$$
y = 1.2 + (2.3f_1) + (-3.4f_2) + (7.12f_3) + (-0.03f_4) + (1.1f_5) + (-0.5f_6) + \varepsilon
$$

The coefficients we want to estimate are:

| Intercept | factor1 | factor2 | factor3 | factor4 | factor5 | factor6 |
|---|---|---|---|---|---|---|
| 1.2 | 2.3 | -3.4 | 7.12 | -0.03 | 1.1 | -0.5 |

The corresponding Julia function is:

In [59]:
# x is a set of flags
function y(x)
    env_flags = ""
    
    for i in 1:length(x)
        if x[i] > 0
            env_flags = "-C passes=" * flags[i] * " " * env_flags
        end
    end
    
    env_flags = "-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals " * env_flags
    
    cmd = `cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`
    
    println(env_flags)
    println(cmd)

    ENV["RUSTFLAGS"] = env_flags

    run(cmd)
    
    exec_time = @elapsed run(`../heap_vec_nolib/target/debug/matrix-multiply-raw`)
    
    return exec_time
end

y (generic function with 1 method)

We can compute the response column for our design using the cell below. Recall that the default is to call the response column `:response`. We are going to set the seeds each time we run `y(x)`, so we analyse same results. Play with different seeds to observe variability of estimates.

In [60]:
Random.seed!(192938)
design.matrix[!, :response] = y.(eachrow(design.matrix[:, collect(design.factors)]))
show(design.matrix, allrows = true, allcols = true)

-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=memcpyopt -C passes=mem2reg -C passes=loop-unswitch -C passes=loop-unroll-and-jam -C passes=loop-unroll -C passes=loop-simplify -C passes=loop-rotate -C passes=loop-reduce -C passes=loop-extract -C passes=loop-deletion -C passes=licm -C passes=lcssa -C passes=jump-threading -C passes=argpromotion -C passes=instcombine -C passes=constprop 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


    Finished dev [unoptimized + debuginfo] target(s) in 0.00s


-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=loop-unswitch -C passes=loop-unroll-and-jam -C passes=loop-unroll -C passes=loop-simplify -C passes=loop-reduce -C passes=loop-deletion -C passes=instcombine -C passes=constprop 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.80s


-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=memcpyopt -C passes=loop-unroll-and-jam -C passes=loop-unroll -C passes=loop-simplify -C passes=loop-rotate -C passes=loop-extract -C passes=licm -C passes=constprop 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


...

-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=memcpyopt -C passes=mem2reg -C passes=loop-unswitch -C passes=loop-unroll-and-jam -C passes=loop-simplify -C passes=loop-reduce -C passes=jump-threading -C passes=argpromotion 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.77s


-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=mem2reg -C passes=loop-unswitch -C passes=loop-unroll-and-jam -C passes=loop-unroll -C passes=loop-rotate -C passes=loop-extract -C passes=argpromotion -C passes=instcombine 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.83s


20×20 DataFrame
│ Row │ factor1 │ factor2 │ factor3 │ factor4 │ factor5 │ factor6 │ factor7 │
│     │ Int64   │ Int64   │ Int64   │ Int64   │ Int64   │ Int64   │ Int64   │
├─────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ 1       │ 1       │ 1       │ 1       │ 1       │ 1       │ 1       │
│ 2   │ 1       │ 1       │ -1      │ -1      │ -1      │ -1      │ 1       │
│ 3   │ 1       │ -1      │ -1      │ -1      │ -1      │ 1       │ -1      │
│ 4   │ -1      │ -1      │ -1      │ -1      │ 1       │ -1      │ 1       │
│ 5   │ -1      │ -1      │ -1      │ 1       │ -1      │ 1       │ -1      │
│ 6   │ -1      │ -1      │ 1       │ -1      │ 1       │ -1      │ 1       │
│ 7   │ -1      │ 1       │ -1      │ 1       │ -1      │ 1       │ 1       │
│ 8   │ 1       │ -1      │ 1       │ -1      │ 1       │ 1       │ 1       │
│ 9   │ -1      │ 1       │ -1      │ 1       │ 1       │ 1       │ 1       │
│ 10  │ 1       │ -1      │ 1       │ 1       │ 

Now, we use the `lm` function from the [GLM package](https://juliastats.org/GLM.jl/stable/) to fit a linear model using the design's matrix and formula:

In [61]:
lm(design.formula, design.matrix)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

response ~ 0 + factor1 + factor2 + factor3 + factor4 + factor5 + factor6 + factor7 + factor8 + factor9 + factor10 + factor11 + factor12 + factor13 + factor14 + factor15 + factor16 + dummy1 + dummy2 + dummy3

Coefficients:
──────────────────────────────────────────────────────────────────────────────
            Estimate  Std. Error       t value  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────────
factor1   -0.228706      20.4124  -0.0112043      0.9929   -259.593    259.136
factor2   -1.53987       20.4124  -0.0754377      0.9521   -260.904    257.824
factor3    0.3097        20.4124   0.0151721      0.9903   -259.055    259.674
factor4   -1.59269       20.4124  -0.0780257      0.9504   -260.957    257.772
factor5   -0.0862522     20.4124  -0.00422548     0.997

The table below shows the coefficients estimated by the linear model fit using the Plackett-Burman Design. The purpose of a screening design is not to estimate the actual coefficients, but instead to compute factor main effects. Note that standard errors are the same for every factor estimate. This happens because the design is orthogonal.

| | Intercept | factor1 | factor2 | factor3 | factor4 | factor5 | factor6 | dummy1 |
|---|---|---|---|---|---|---|---|---|
| Original | 1.2 | 2.3 | -3.4 | 7.12 | -0.03 | 1.1 | -0.5 | $-$ |
| Plackett-Burman Main Effects | $-$ | 2.66359 | -2.80249 | 7.71644 | -0.0497774 | 0.612681 | -0.112675 | -0.437176 |

We can use the coefficient magnitudes to infer that factor 3 probably has a strong main effect, and that factor 6 has not. Our dummy column had a relatively small coefficient estimate, so we could attempt to ignore interactions on subsequent experiments.

# Fitting a Linear Model

We can also try to fit a linear model on our design data in order to estimate coefficients. We would need to drop the dummy column and add the intercept term:

In [62]:
lm(term(:response) ~ sum(term.(design.factors)), design.matrix)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

response ~ 1 + factor1 + factor2 + factor3 + factor4 + factor5 + factor6 + factor7 + factor8 + factor9 + factor10 + factor11 + factor12 + factor13 + factor14 + factor15 + factor16

Coefficients:
────────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error      t value  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────────
(Intercept)  20.4124       0.133477  152.928        <1e-6   19.9876    20.8372
factor1      -0.228706     0.133477   -1.71344      0.1851  -0.65349    0.196079
factor2      -1.53987      0.133477  -11.5365       0.0014  -1.96465   -1.11508
factor3       0.3097       0.133477    2.32024      0.1031  -0.115085   0.734484
factor4      -1.59269      0.133477  -11.9323       0.0013  -2.01748   

Our table so far looks like this:

| | Intercept | factor1 | factor2 | factor3 | factor4 | factor5 | factor6 | dummy1 |
|---|---|---|---|---|---|---|---|---|
| Original | 1.2 | 2.3 | -3.4 | 7.12 | -0.03 | 1.1 | -0.5 | $-$ |
| Plackett-Burman Main Effects | $-$ | 2.66359 | -2.80249 | 7.71644 | -0.0497774 | 0.612681 | -0.112675 | -0.437176 |
| Plackett-Burman Estimate | 0.940183 | 2.66359 | -2.80249 | 7.71644 | -0.0497774 | 0.612681 | -0.112675 | $-$ |

Notice that, since the standard errors are the same for all factors, factors with stronger main effects are better estimated. Notice that, despite the "good" coefficient estimates, the confidence intervals are really large.

This is a biased comparison where the screening design "works" for coefficient estimation as well, but we would rather use fractional factorial or optimal designs to estimate the coefficients of factors with strong effects. Screening should be used to compute main effects and identifying which factors to test next.

# Generating Random Designs

We can also compare the coefficients produced by the same linear model fit, but using a random design. For more information, check [the docs](https://phrb.github.io/ExperimentalDesign.jl/dev/lib/public/#ExperimentalDesign.RandomDesign-Tuple{NamedTuple}).

In [63]:
Random.seed!(8418172)

random_design_generator = RandomDesign(DiscreteNonParametric([-1, 1], [0.5, 0.5]), 6)
random_design = rand(random_design_generator, 8)

random_design[!, :response] = y.(eachrow(random_design[:, :]))
random_design

-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=lcssa -C passes=instcombine 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.76s


-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=argpromotion 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.74s


-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=licm -C passes=lcssa -C passes=jump-threading -C passes=instcombine 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


...

-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=argpromotion 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.79s


-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=lcssa -C passes=jump-threading -C passes=argpromotion 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.77s


,factor1,factor2,factor3,factor4,factor5,factor6,response
,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,-1,1,-1,-1,1,-1,20.202
2,-1,-1,1,-1,-1,-1,25.5836
3,-1,1,-1,1,1,1,17.8959
4,-1,-1,1,1,1,1,22.5354
5,-1,1,1,-1,1,-1,20.1528
6,1,-1,1,-1,-1,-1,25.2826
7,-1,-1,1,-1,-1,-1,25.3227
8,-1,-1,1,1,1,-1,22.0858


In [64]:
lm(random_design_generator.formula, random_design)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

response ~ 1 + factor1 + factor2 + factor3 + factor4 + factor5 + factor6

Coefficients:
──────────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error     t value  Pr(>|t|)  Lower 95%     Upper 95%
──────────────────────────────────────────────────────────────────────────────────
(Intercept)  21.5893       0.130491  165.447       0.0038   19.9312   23.2473
factor1      -0.0852702    0.113008   -0.754549    0.5885   -1.52117   1.35063
factor2      -2.34439      0.184541  -12.7039      0.0500   -4.68921   0.000431755
factor3      -0.0246326    0.130491   -0.188769    0.8812   -1.68267   1.63341
factor4      -1.37786      0.184541   -7.46638     0.0848   -3.72268   0.966965
factor5      -0.305796     0.172623   -1.77147     0.3272   -2.49918   1.88758
factor6       0

Now, our table looks like this:

| | Intercept | factor1 | factor2 | factor3 | factor4 | factor5 | factor6 | dummy1 |
|---|---|---|---|---|---|---|---|---|
| Original | 1.2 | 2.3 | -3.4 | 7.12 | -0.03 | 1.1 | -0.5 | $-$ |
| Plackett-Burman Main Effects | $-$ | 2.66359 | -2.80249 | 7.71644 | -0.0497774 | 0.612681 | -0.112675 | -0.437176 |
| Plackett-Burman Estimate | 0.940183 | 2.66359 | -2.80249 | 7.71644 | -0.0497774 | 0.612681 | -0.112675 | $-$ |
| Single Random Design Estimate | 0.600392 | 2.2371 | -2.56857 | 8.05743 | 0.140622 | 0.907918 | -0.600354 | $-$ |

The estimates produced using random designs will have larger confidence intervals, and therefore increased variability. The Plackett-Burman design is fixed, but can be randomised. The variability of main effects estimates using screening designs will depend on measurement or model error.

# Generating Full Factorial Designs

In this toy example, it is possible to generate all the possible combinations of six binary factors and compute the response. Although it costs 64 experiments, the linear model fit for the full factorial design should produce the best coefficient estimates.

The `explicit` parameter below makes the full factorial a complete `DataFrame` in memory. Since factorial designs can be prohibitively large, you can omit the `explicit` parameter, or set it to `false`, to create an iterator that will generate design lines on demand.

The simplest full factorial design constructor receives an array of possible factor levels. For more, check [the docs](https://phrb.github.io/ExperimentalDesign.jl/dev/lib/public/#ExperimentalDesign.FullFactorial-Tuple{NamedTuple,StatsModels.FormulaTerm}).

In [65]:
Random.seed!(2989476)

factorial_design = FullFactorial(fill([-1, 1], 6), explicit = true)
factorial_design.matrix[!, :response] = y.(eachrow(factorial_design.matrix[:, :]))

lm(factorial_design.formula, factorial_design.matrix)

-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.74s


-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=constprop 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.74s


-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=instcombine 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


...

-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=lcssa 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.68s


-C lto=off -C no-prepopulate-passes -C passes=name-anon-globals -C passes=lcssa -C passes=constprop 
`cargo build --manifest-path ../heap_vec_nolib/Cargo.toml`


   Compiling libc v0.2.69
   Compiling getrandom v0.1.14
   Compiling cfg-if v0.1.10
   Compiling ppv-lite86 v0.2.6
   Compiling rand_core v0.5.1
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3
   Compiling matrix-multiply-raw v0.1.0 (/home/emanuel/Documentos/Git/matrix-multiply-test/heap_vec_nolib)
    Finished dev [unoptimized + debuginfo] target(s) in 2.76s


ProcessFailedException: failed process: Process(`../heap_vec_nolib/target/debug/matrix-multiply-raw`, ProcessSignaled(2)) [0]


The confidence intervals for this fit are much smaller. Since we have all information on all factors and this is a balanced design, the standard error is the same for all estimates. Here's the complete table:

| | Intercept | factor1 | factor2 | factor3 | factor4 | factor5 | factor6 | dummy1 |
|---|---|---|---|---|---|---|---|---|
| Original | 1.2 | 2.3 | -3.4 | 7.12 | -0.03 | 1.1 | -0.5 | $-$ |
| Plackett-Burman Main Effects | $-$ | 2.66359 | -2.80249 | 7.71644 | -0.0497774 | 0.612681 | -0.112675 | -0.437176 |
| Plackett-Burman Estimate | 0.940183 | 2.66359 | -2.80249 | 7.71644 | -0.0497774 | 0.612681 | -0.112675 | $-$ |
| Single Random Design Estimate | 0.600392 | 2.2371 | -2.56857 | 8.05743 | 0.140622 | 0.907918 | -0.600354 | $-$ |
| Full Factorial Estimate | 1.13095 | 2.23668 | -3.4775 | 6.95531 | -0.160546 | 0.975471 | -0.357748 | $-$ |

Full factorial designs may be too expensive in actual applications. Fractional factorial designs or optimal designs can be used to decrease costs while still providing good estimates. Screening designs are extremely cheap, and can help determine which factors can potentially be dropped on more expensive and precise designs.

Check the examples directory for more tutorials!